## Item Asset Activation and Download
In order to download imagery assets, they must first be activated. Once active, the asset may be downloaded using the provided link.

 * Full resolution uncompressed satellite images are big and there are lots of ways to view them.
 * For this reason Planet generally keeps images in their native format and only processes them on customer requests.
 * All images must be activated prior to downloading and this can take some time based on demand.
 * Generally speaking there are three kinds of scenes:
    * Analytic - multi-band full resolution images that have not been processed. These are like raw files for DSLR camers.
    * Visual - these are color corrected rectified tifs. If you are just starting out this is your best call.
    * UDM - Usable data mask. This mask can be used to find bad pixels and columns and to mask out areas with clouds.

### Input Parameters 

In [ ]:
from os import environ

# Set API key (this should to be an environment variable)
api_key = environ['PLANET_API_KEY']

# Sent Scene ID
scene_id = '20170604_180819_0f52'

# Set Item Type
item_type = 'PSScene4Band'

# Set Asset Type
asset_type = 'analytic'

### Activate Item Assets provided above

In [ ]:
from planet import api
import time

# Create Planet API client object
client = api.ClientV1(api_key)

# Activate the item asset (if nessesary) and wait for activation to complete
asset_activated = False
while asset_activated == False:
    
    # Get asset and its activation status
    all_assets = client.get_assets_by_id(item_type, scene_id)  
    asset = all_assets.get()[asset_type]
    asset_status = asset["status"]
    
    # If asset is already active, we are done
    if asset_status == 'active':
        asset_download_url = asset['location']
        asset_activated = True
        print("Asset is active and ready to download")
    
    # If asset is inactive, we must send an activation request and wait for activation to complete
    elif asset_status == 'inactive':
        client.activate(asset)
        print("Asset activation request sent. Waiting for activation.")
    
    # Still activating. Wait 1 second and check again.
    else:
        print("...Still waiting for asset activation...")
        time.sleep(1)

### Once Item is Active, it can be downloaded

In [ ]:
out = client.download(asset)
file_body = out.get_body()
file_body.write('../data_storage/' + file_body.name)

print('Asset ../data_storage/' + file_body.name + ' successfully downloaded.')